## Part 1 - Closest to (0,0,0) in the long run

In [1]:
import numpy as np
def load(file_name):
    '''Loads positions, velocities, and accelerations (as tuples of ints) from the file_name'''
    file = open(file_name, 'r')
    poss = []
    vels = []
    accs = []
    for s in file:
        poss.append(tuple(int(n) for n in s[s.find('p=<')+3:s.find('>, v=<')].split(',')))
        vels.append(tuple(int(n) for n in s[s.find('v=<')+3:s.find('>, a=<')].split(',')))
        accs.append(tuple(int(n) for n in s[s.find('a=<')+3:s.rfind('>')].split(',')))
    return np.array(poss), np.array(vels), np.array(accs)

In [2]:
poss, vels, accs = load('in/day20.txt')
acc_mds = np.abs(accs).sum(axis=-1)  # Absolute values of accelerations of each particle
min_abs_acc = np.where(acc_mds == acc_mds.min())[0]  # Array of indexes with absolute acceleration = min
# If there were more than 1 such particles - initial velocity should have been considered.
print('Answer to part 1 - particle whose acceleration is the lowest = {}'.format(min_abs_acc))

Answer to part 1 - particle whose acceleration is the lowest = [150]


## Part 2 - How many particles collided?

In [3]:
p, v, a = np.array(poss), np.array(vels), np.array(accs)
print('Starting similation with {} particles'.format(len(p)))
from mpl_toolkits.mplot3d import Axes3D 
import matplotlib.pyplot as plt
for t in range(100):
    #print('t = {}, {}, {}, {}'.format(t, p[-1], v[-1], a[-1]))
    unique, counts = np.unique(p, axis=0, return_counts=True)
    collision_mask = counts > 1
    if True in collision_mask:  # Collisions (non-unique positions) found
        collision_poss = unique[collision_mask]  # Duplicated positions (collisions) to be removed
        collision_idx = [i for i in range(len(p)) if p[i] in collision_poss]
        #print('----------\nt = {}, particles count = {}, collisions: {}, collision positions:\n{}'.format(
            #t, len(p), len(collision_idx), p[collision_idx]))
        #fig = plt.figure()
        #ax = fig.add_subplot(111, projection='3d') 
        #ax.scatter(p[:,0], p[:,1], p[:,2], s=1, c="g", marker="o")
        #ax.scatter(collision_poss[:,0], collision_poss[:,1], collision_poss[:,2], s=50, c="r", marker="o")
        #ax.view_init(azim=180, elev=10)
        plt.show()

        p = np.delete(p, collision_idx, axis=0)
        v = np.delete(v, collision_idx, axis=0)
        a = np.delete(a, collision_idx, axis=0)
        print('t = {}, after collisions remain {} particles'.format(t, len(p)))
    v += a
    p += v
print('----------\nPart 2 answer: {}'.format(len(p)))

Starting similation with 1000 particles
t = 10, after collisions remain 979 particles
t = 11, after collisions remain 972 particles
t = 12, after collisions remain 956 particles
t = 13, after collisions remain 948 particles
t = 14, after collisions remain 933 particles
t = 16, after collisions remain 921 particles
t = 17, after collisions remain 896 particles
t = 18, after collisions remain 881 particles
t = 20, after collisions remain 864 particles
t = 21, after collisions remain 840 particles
t = 23, after collisions remain 831 particles
t = 24, after collisions remain 820 particles
t = 25, after collisions remain 797 particles
t = 26, after collisions remain 780 particles
t = 27, after collisions remain 767 particles
t = 28, after collisions remain 754 particles
t = 29, after collisions remain 745 particles
t = 30, after collisions remain 733 particles
t = 31, after collisions remain 728 particles
t = 33, after collisions remain 716 particles
t = 34, after collisions remain 707 part

### Part 2 answers
Too high: `772`.

Incorrect: `644`, `663`.

Correct: `657`

# Someone elses' solution
[Here](https://gist.github.com/GlenboLake/91fa9b990e46b6b624704e9a1c7495c8)

In [4]:
import re
from cmath import sqrt
from collections import defaultdict, namedtuple
from functools import reduce
from itertools import combinations
from time import time

Particle = namedtuple('Particle', ['pos', 'vel', 'acc'])

def parse_particle(line):
    pos_match = re.search('p=<(-?\d+),(-?\d+),(-?\d+)>', line)
    position = (int(pos_match.group(1)), int(pos_match.group(2)), int(pos_match.group(3)))
    vel_match = re.search('v=<(-?\d+),(-?\d+),(-?\d+)>', line)
    velocity = (int(vel_match.group(1)), int(vel_match.group(2)), int(vel_match.group(3)))
    acc_match = re.search('a=<(-?\d+),(-?\d+),(-?\d+)>', line)
    acceleration = (int(acc_match.group(1)), int(acc_match.group(2)), int(acc_match.group(3)))
    return Particle(position, velocity, acceleration)

def particle_at(particle, t):
    x = particle.pos[0] + particle.vel[0] * t + particle.acc[0] * t * (t + 1) // 2
    y = particle.pos[1] + particle.vel[1] * t + particle.acc[1] * t * (t + 1) // 2
    z = particle.pos[2] + particle.vel[2] * t + particle.acc[2] * t * (t + 1) // 2
    return x, y, z

def manhattan(point):
    return sum(map(abs, point))

def part1(particles):
    max_accel = max(sum(map(abs, p.acc)) for p in particles)
    return particles.index(min(particles, key=lambda p: manhattan(particle_at(p, 100 * max_accel))))

def will_collide(p1, p2):
    def is_int(c):
        return c.imag == 0 and (isinstance(c.real, int) or c.real.is_integer())

    def solve_quadratic(a, b, c):
        solutions = None
        if a:
            solutions = {(-b - sqrt(b ** 2 - 4 * a * c)) / (2 * a), (-b + sqrt(b ** 2 - 4 * a * c)) / (2 * a)}
        elif b:
            solutions = {-c / b}
        elif c:
            solutions = {c}
        if solutions is not None:
            solutions = set(map(lambda x: int(x.real), filter(is_int, solutions)))
        return solutions

    diff = Particle(tuple(a - b for a, b in zip(p1.pos, p2.pos)),
                    tuple(a - b for a, b in zip(p1.vel, p2.vel)),
                    tuple(a - b for a, b in zip(p1.acc, p2.acc)))
    tuples = [
        (diff.acc[0], diff.vel[0], diff.pos[0]),
        (diff.acc[1], diff.vel[1], diff.pos[1]),
        (diff.acc[2], diff.vel[2], diff.pos[2]),
    ]
    solutions = reduce(lambda a, b: a & b,
                       filter(lambda s: s is not None,
                              [solve_quadratic(a / 2, v + a / 2, p) for a, v, p in tuples]))

    if solutions:
        return min(s for s in solutions if s > 0)
    return None

def pairs_to_sets(data):
    items = {a for a, b in data} | {b for a, b in data}
    sets = []
    seen = set()
    for item in items:
        if item in seen:
            continue
        new_set = set()
        seen.add(item)
        for pair in data:
            if item in pair:
                seen.update(set(pair))
                new_set.update(set(pair))
        sets.append(new_set)
    return sets

def part2(particles):
    collisions = defaultdict(list)
    for a, b in combinations(particles, 2):
        t = will_collide(a, b)
        if t is not None:
            collisions[t].append({a, b})
    collisions = {k: pairs_to_sets(v) for k, v in collisions.items()}
    remaining = set(particles)
    for t, splosions in sorted(collisions.items()):
        for s in splosions:
            if len(remaining & s) > 1:
                remaining -= s
    return len(remaining)

if __name__ == '__main__':
    particles = [parse_particle(line) for line in open('in/day20.txt')]

    start = time()
    print('Part 1:', part1(particles))
    print('Part 2:', part2(particles))
    print(f'Took {time()-start}s')

Part 1: 150
Part 2: 657
Took 6.849648952484131s
